In [1]:
import pandas as pd
import re
import json
import MyData as md
import torch
from layer import InputEmbed as IE
import MyTrans
import utils

In [2]:
with open("data/train1.json", 'r', encoding= 'utf-8') as file:
    data = json.load(file)
# with open("data/train.json", "r", encoding='utf-8') as file:
#     train = json.load(file)
# with open("data/test.json", "r", encoding='utf-8') as file:
#     test = json.load(file)
# with open("data/valid.json", "r", encoding='utf-8') as file:
#     valid = json.load(file)    
    

In [2]:
with open("vocab/vocabE1.json", "r", encoding='utf-8') as file:
    vocabE = json.load(file)  
with open("vocab/vocabV1.json", "r", encoding='utf-8') as file:
    vocabV = json.load(file)  
    
E_tokenizer = md.MyTokenizer(vocabE)
V_tokenizer = md.MyTokenizer(vocabV)    

In [3]:
data = md.DataLoader(md.EV_Data(data_path= "data/test1.json", E_vocab_path= "vocab/vocabE1.json", V_vocab_path= "vocab/vocabV1.json", inp='V', out= 'E'), batch_size= 2)

In [4]:
results = torch.load("checkpoint/bestmodel_parallel.pth", map_location=torch.device('cpu'))
model = MyTrans.TransformerParallel(input_vocab_size= len(vocabE), output_vocab_size= len(vocabV), dmodel = 128, dembed = 128, d_ff = 256, head = 8, layer = 2, dropout = 0.1,
                                    eps = 1e-5)
model.load_state_dict(results['model'])


<All keys matched successfully>

In [6]:
a = next(data._get_iterator())

In [7]:
b = model.inference(a[0], 0, 300)

In [12]:
E_tokenizer.decode(b[1])

"<start> tom won't be here tomorrow . i am sure it's going to be sure i'm not going to be here tomorrow . i'm sure i'm sure i'm sure i'm sure i'm sure i'm sure i'm going to make sure tomorrow . tomorrow . i'm going to be sure to make sure i'm not going to be sure tomorrow . i'm not tomorrow . i'm sure i'm sure it's going to be sure i'm sure i'm going to be sure i'm going to be sure i'm going to make sure tomorrow . tomorrow . i'm sure tomorrow . i'm sure tomorrow . i'm sure i'm sure i'm sure i'm sure tomorrow . i'm going to make sure to make sure i'm sure tomorrow . i'm sure i'm sure i'm sure i'm sure i'm sure tomorrow . i'm sure i'm sure i'm sure . i sure i'm sure . i'm not going to be sure i'm sure i'm sure i'm going to make sure tomorrow . i'll make sure tomorrow . i'm sure i'm not tomorrow . i'm sure i'm sure i'm sure i'm sure i'm going to make sure i'm going to make sure i'm sure tomorrow . i'm sure i'm sure i'm going to make sure i'm sure it's sure . i sure i'm sure . tomorrow . 

In [6]:
E_tokenizer.decode(a[1][1])

NameError: name 'a' is not defined

In [7]:
data = "tom won't be here tomorrow . i'm sure to tell you that"
utils.predict_parallel(model, data, E_tokenizer, V_tokenizer, 'E')

'<start> tom sẽ không ở đây vào ngày mai tôi chắc chắn nói với bạn rằng <end>'

AttributeError: 'generator' object has no attribute 'sum'

In [2]:
with open("data/train.json", "r", encoding='utf-8') as file:
    train = json.load(file)
with open("data/test.json", "r", encoding='utf-8') as file:
    test = json.load(file)
with open("data/valid.json", "r", encoding='utf-8') as file:
    valid = json.load(file)
with open("data/train1.json", "r", encoding='utf-8') as file:
    train1 = json.load(file)
with open("data/test1.json", "r", encoding='utf-8') as file:
    test1 = json.load(file)
with open("data/valid1.json", "r", encoding='utf-8') as file:
    valid1 = json.load(file)    


In [3]:
with open("data/train1.json", "r", encoding='utf-8') as file:
    train1 = json.load(file)
with open("data/valid1.json", "r", encoding='utf-8') as file:
    valid1 = json.load(file)
with open("data/test1.json", "r", encoding='utf-8') as file:
    test1 = json.load(file)        
    

In [30]:
my = [train1, valid1, test1]
for name in my:    
    for i in range(len(name)):
        name[i]['E'] = re.sub(r'^(en:|vi:)', '', name[i]['E'])
        name[i]['V'] = re.sub(r'^(en:|vi:)', '', name[i]['V'])

In [31]:
with open("data/valid1.json", 'w', encoding='utf-8') as f:
    json.dump(train1, f, ensure_ascii=False, indent=4)

In [1]:
BOS = '<start>'
EOS = '<end>'
en = "en: "
vi = "vi: "
pattern = r'([^a-zA-Z0-9])'  
def pre_tokenize(w):
    w = re.sub(pattern, r' \1 ', w)         
    w = w.lower().strip()
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'(\d)', r' \1 ', w)
    w = re.sub(r'\\n', '', w)
    w = re.sub(r'\s*:\s*', ' : ', w)
    w = re.sub(r'(["\'])', r' \1 ', w)    
    w = re.sub(r'\\', ' ', w)
    w = re.sub(r'-{2,}', '-', w)  # Thay thế 2 hoặc nhiều dấu gạch ngang liên tiếp bằng 1 dấu gạch ngang
    w = re.sub(r'-+', ' - ', w)  # Thêm khoảng trắng xung quanh dấu gạch ngang hoặc chuỗi dấu gạch ngang
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r'\s+', ' ', w)  # Loại bỏ khoảng trắng thừa
    w = w.strip()
    # Add start and end token 
    w = '{} {} {}'.format(BOS, w, EOS)

In [16]:
name = [train, test, valid]
for my in name:
    for i in range(len(my)):
        my[i]['E'] = f"{en}{my[i]['E']}" 
        my[i]['V'] = f"{vi}{my[i]['V']}"

In [25]:
train = train + train1
test = test + test1
valid = valid + valid1


In [36]:
name = [train1, test1, valid1]
vocabE = {}
vocabV = {}
vocabE[' '] = 0
vocabV[' '] = 0
k = 1
h = 1
for my in name:
    for i in range(len(my)):
        e = preprocess_sentence(my[i]['E'])
        v = preprocess_sentence(my[i]['V']) 
        for w in e.split(sep = ' '):
            if w not in vocabE:
                vocabE[w] = k
                k = k+1
        for w in v.split(sep = ' '):
            if w not in vocabV:
                vocabV[w] = h
                h = h+1        

In [37]:
preprocess_sentence('----it').split(sep = ' ')

['<start>', '-', 'it', '<end>']

In [40]:
len(vocabV)

21913

In [59]:
with open('data/test1.json', 'w', encoding='utf-8') as f:
    json.dump(test, f, ensure_ascii=False, indent=4)

In [16]:
with open("vocab/vocabE2.json", 'w', encoding='utf-8') as f:
    json.dump(vocabE, f, ensure_ascii=False, indent=4)

In [6]:
data = md.EV_Data('data/train.json', 'vocab/vocab_E.json', 'vocab/vocab_V.json')

In [40]:
a = torch.randn(size = [4,4], requires_grad = True, dtype = torch.float32)
index = torch.ones(size = [4, 2,2], dtype= torch.int64)

In [41]:
torch.gather(a.repeat(4,1,1), dim = -1, index = index)

tensor([[[-0.0881, -0.0881],
         [-1.1430, -1.1430]],

        [[-0.0881, -0.0881],
         [-1.1430, -1.1430]],

        [[-0.0881, -0.0881],
         [-1.1430, -1.1430]],

        [[-0.0881, -0.0881],
         [-1.1430, -1.1430]]], grad_fn=<GatherBackward0>)